# EdelweissData Query Language

This notebook shows the python client library equivalents to the raw HTTP calls described in the [query language](https://edelweissdata.com/docs/query-language) walkthrough on the offical EdelweissData documentation website.

EdelweissData™ has a rich Query language that allows you to filter and sort both queries to find datasets as well as queries for the actual data of an individual dataset.

To make it easy to get familiar with the Query Language this walkthrough is divided into two sections: an overview of the Query language and then an enumeration of the different types of expressions with examples.

A useful way to get familiar with the Query Language is also to view a dataset in the [EdelweissData™ DataExplorer](https://edelweissdata.com/dataset/8dde2785-8a2a-4847-80b8-982a691954d6:128?q=%7B%7D), filter the data by filling search fields in the UI and ordering it by clicking the icons and then clicking the "API" button in the top right corner to copy/paste the code including the Query expression to filter & order data rows just like you see it in the DataExplorer.

## API initialization

(See the [setup notebook](setup.ipynb) for details on how to install, initialize and authorize the library)

In [1]:
from edelweiss_data import API, QueryExpression as Q

# Set this to the url of the Edelweiss Data server you want to interact with
edelweiss_api_url = 'https://api.edelweissdata.com'

api = API(edelweiss_api_url)

In [2]:
# Setup to be able to query the Our world in Data dataset:
datasetid = "b55b229d-6338-4e41-a507-0cf4d3297b54"
version = 117

dataset = api.get_published_dataset(datasetid, version)

In [3]:
import altair

## Overview

The EdelweissData python client API provides convenience methods to construct the various parts of the query language and takes care of translating it correctly to the underlying JSON serialisation. As an example, this is how to filter rows in a datasets to only those that have the text value "Germany" in the column called "location" (see below for the results of running this query on a real dataset).

Note that when importing the edelweiss_data library, the QueryExpression class that contains all the method helpers is often aliased to Q because it is used a lot

In [4]:
from edelweiss_data import API, QueryExpression as Q

filter = Q.exact_search(Q.column("location"), "Germany")

## Applications

The Query Language can be used with EdelweissData™ API Endpoints that queries for datasets ([/datasets](https://api.edelweissdata.com/docs/index.html#operations-Published-getPublishedDatasetsViaPost)) as well as those that query for data of an individual dataset ([/datasets/{datasetId}/versions/{version}/data](https://api.edelweissdata.com/docs/index.html#operations-Published-postPublishedDatasetData)). In both cases it can be used to specify the sorting of the data and to filter rows. In the sorting case the Expression has to evaluate to a numeric or textual value (i.e. not a boolean), for the filtering the expression has to evaluate to a boolean value (i.e. whether or not to include a row).

In the interactive examples below we will have two query targets. For querying the data of a dataset we will use the COVID-19 data compiled by [Our world in data](https://ourworldindata.org), filtered to various countries etc. You can look at the dataset in the browser in the [EdelweissData™ DataExplorer](https://edelweissdata.com/dataset/b55b229d-6338-4e41-a507-0cf4d3297b54:117?q=%7B%7D). For the datasets query we will query for all public datasets at [edelweissdata.com](https://edelweissdata.com/datasets)

# Elements of the Query Language

## Values

Values are the simplest elements of a Query and are used verbatim (i.e. there is no special escaping etc necessary, they are simply JSON values). You can either use string values (in quotes, e.g. `"Germany"`), numbers (e.g. `4` or `3.2` or `2.384e12`), booleans (`True` or `False`), arrays (python arrays, i.e. `[2.0, 3.0, 4.0]`) or objects (python dicts, e.g. `{"some-key": "some-value"}`). Of these, strings and numbers are by far used the most. Arrays can be useful for contains queries (where you want to know if a column of an array data type contains the values of the given array). Objects are only used occasionally when doing queries for datasets that include parts of the metadata that can of course contain nested objects.

## Columns

Column expression reference a given column by name and return the value in the column of the given name. Columns are expressions can be constructed with the QueryExpression helper `QueryExpression.column()`:

In [5]:
filter = Q.exact_search(Q.column("location"), "Germany")
data_for_germany = dataset.get_data(condition = filter)
data_for_germany

,,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
9152,9151,DEU,Europe,Germany,2019-12-31,0,0,NaN,0,0,...,15.957,45229.245,None,156.139,8.31,28.2,33.1,None,8,81.33
9153,9152,DEU,Europe,Germany,2020-01-01,0,0,NaN,0,0,...,15.957,45229.245,None,156.139,8.31,28.2,33.1,None,8,81.33
9154,9153,DEU,Europe,Germany,2020-01-02,0,0,NaN,0,0,...,15.957,45229.245,None,156.139,8.31,28.2,33.1,None,8,81.33
9155,9154,DEU,Europe,Germany,2020-01-03,0,0,NaN,0,0,...,15.957,45229.245,None,156.139,8.31,28.2,33.1,None,8,81.33
9156,9155,DEU,Europe,Germany,2020-01-04,0,0,NaN,0,0,...,15.957,45229.245,None,156.139,8.31,28.2,33.1,None,8,81.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9385,9384,DEU,Europe,Germany,2020-08-20,228621,1707,1128.143,9253,10,...,15.957,45229.245,None,156.139,8.31,28.2,33.1,None,8,81.33
9386,9385,DEU,Europe,Germany,2020-08-21,230948,2327,1253.143,9260,7,...,15.957,45229.245,None,156.139,8.31,28.2,33.1,None,8,81.33
9387,9386,DEU,Europe,Germany,2020-08-22,232082,1134,1223.857,9267,7,...,15.957,45229.245,None,156.139,8.31,28.2,33.1,None,8,81.33
9388,9387,DEU,Europe,Germany,2020-08-23,232864,782,1233.000,9269,2,...,15.957,45229.245,None,156.139,8.31,28.2,33.1,None,8,81.33


In [6]:
altair.Chart(data_for_germany).mark_circle().encode(
        x="date:T",
        y="new_cases",
        color="location",
        tooltip=["location", "date", "new_cases", "new_deaths"]
    ).properties(width=1000)

alt.Chart(...)

Queries that query for datasets can use JSONpaths to map fragments of the metadata into new columns with names supplied with the JSONpath. To distinguish between always existing columns like the dataset name (`name`) or the creation timestamp (`created`) and user defined column names, there exist the special `QueryExpression.system_column()` method that is used for system specified columns, e.g. to search for a dataset with the name "COVID-19 complete dataset by Our World In Data" the following expression would be used to indicate that we want to search in the system defined name column, not a column with the same name that was constructed with a JSONpath query:

In [7]:
filter_datasets_by_name = Q.exact_search(Q.system_column("name"), "COVID-19 complete dataset by Our World In Data")

In [8]:
datasets = api.get_published_datasets(condition = filter_datasets_by_name)
datasets

,,dataset
id,version,
b55b229d-6338-4e41-a507-0cf4d3297b54,143,<PublishedDataset 'b55b229d-6338-4e41-a507-0cf...


In [9]:
datasets.iloc[0].dataset.name

'COVID-19 complete dataset by Our World In Data'

## Text searches

Three text search functions exist in EdelweissData™: `Q.search_anywhere()` which searches in any text-like column;`Q.exact_search()` which searches in a given column for an exact string value; and `Q.fuzzy_search()` which does a more lenient search within a given column for a string (matchings substrings). Text searches result in a boolean value.

### Search anywhere

In [10]:
filter = Q.search_anywhere("United")

data_united = dataset.get_data(condition = filter)

altair.Chart(data_united).mark_circle().encode(
        x="date:T",
        y="new_cases",
        color="location",
        tooltip=["location", "date", "new_cases", "new_deaths"]
    ).properties(width=1000)

alt.Chart(...)

### Exact search

In [11]:
filter = Q.exact_search(Q.column("location"), "United States")

data_united_states = dataset.get_data(condition = filter)

altair.Chart(data_united_states).mark_circle().encode(
        x="date:T",
        y="new_cases",
        color="location",
        tooltip=["location", "date", "new_cases", "new_deaths"]
    ).properties(width=1000)

alt.Chart(...)

### Fuzzy search

In [12]:
filter = Q.fuzzy_search(Q.column("location"), "States")

data_states = dataset.get_data(condition = filter)

altair.Chart(data_states).mark_circle().encode(
        x="date:T",
        y="new_cases",
        color="location",
        tooltip=["location", "date", "new_cases", "new_deaths"]
    ).properties(width=1000)

alt.Chart(...)

## Relations

Relations express the usual comparisons and in the EdelweissData python library are modelled as infix operators: equality (`==`), greaterThan (`>`), greaterThanOrEqual(`>=`), lessThan (`<`), lessThanOrEqual(`<=`) and unequal(`!=`). All of these take two arguments that are then compared with the result being a boolean value.

In [13]:
filter = (Q.column("new_cases") == 100)

data_100_cases = dataset.get_data(condition = filter)

altair.Chart(data_100_cases).mark_circle().encode(
        x="date:T",
        y="new_cases",
        color="location",
        tooltip=["location", "date", "new_cases", "new_deaths"]
    ).properties(width=1000)

alt.Chart(...)

In [14]:
filter = (Q.column("new_cases") > 50000)

data_above_50000 = dataset.get_data(condition = filter)

altair.Chart(data_above_50000).mark_circle().encode(
        x="date:T",
        y="new_cases",
        color="location",
        tooltip=["location", "date", "new_cases", "new_deaths"]
    ).properties(width=1000)

alt.Chart(...)

#### Contains/ContainedIn

In addition to the usual suspects above there exist two special relations, `Q.contains` and `Q.contained_in` that check if the first argument is contains the second (`contains`) or vice versa (`contained_in`)

In [15]:
filter_by_keyword = Q.contains(Q.column("keywords"), "covid-19")

datasets = api.get_published_datasets(condition = filter_by_keyword, columns = [("keywords", "$.keywords[*]", "xsd:string")])
datasets

,,dataset,keywords
id,version,,
8dde2785-8a2a-4847-80b8-982a691954d6,131,<PublishedDataset '8dde2785-8a2a-4847-80b8-982...,"[covid-19, cases, deaths, Germany]"
dade0fa0-782d-47fa-84bd-9e3388bf9580,134,<PublishedDataset 'dade0fa0-782d-47fa-84bd-9e3...,"[covid-19, cases, deaths, by country]"
21842e38-ad7d-454f-9bfb-e58e79ffdf8b,137,<PublishedDataset '21842e38-ad7d-454f-9bfb-e58...,"[covid-19, cases, deaths, by country]"
87f93af6-fd7a-4c25-961a-07e53fb97250,137,<PublishedDataset '87f93af6-fd7a-4c25-961a-07e...,"[covid-19, cases, deaths, by country]"
f67ffafd-702b-4705-b33d-8258f5090b57,138,<PublishedDataset 'f67ffafd-702b-4705-b33d-825...,"[covid-19, cases, deaths, by country]"
72421b77-78e8-4fad-9e3b-33956585e515,137,<PublishedDataset '72421b77-78e8-4fad-9e3b-339...,"[covid-19, cases, deaths, USA]"
b55b229d-6338-4e41-a507-0cf4d3297b54,143,<PublishedDataset 'b55b229d-6338-4e41-a507-0cf...,"[covid-19, cases, deaths, by country, testing]"


## Logical operators

The usual logical operators are supported - because python allows to override the bitwise variants but not the logical operators, you have to use the infix bitwise not, and and or variants: not (`~` operator), and (`&` operator) and or (`|` operator). All require boolean arguments and evaluate in turn to a boolean value. 

In [16]:
filter = (~ (Q.column("new_cases") < 50000))

data_above_50000 = dataset.get_data(condition = filter)

altair.Chart(data_above_50000).mark_circle().encode(
        x="date:T",
        y="new_cases",
        color="location",
        tooltip=["location", "date", "new_cases", "new_deaths"]
    ).properties(width=1000)

alt.Chart(...)

In [17]:
filter = (Q.column("new_cases") > 2000) & (Q.exact_search(Q.column("location"), "Italy") | Q.exact_search(Q.column("location"), "Germany"))

data_above_2000_germany_or_italy = dataset.get_data(condition = filter)

altair.Chart(data_above_2000_germany_or_italy).mark_circle().encode(
        x="date:T",
        y="new_cases",
        color="location",
        tooltip=["location", "date", "new_cases", "new_deaths"]
    ).properties(width=1000)

alt.Chart(...)

## Special functions

There are a few additional capabilities that fall outside the range of the usual query vocabulary. These revolve around specific domain types, notably the SMILES column type for the chemical structure notation of the same name.

### Tanimoto Similarity

`tanimoto_similarity` allows you to calculate the [Tanimoto Similarity](https://en.wikipedia.org/wiki/Jaccard_index#Tanimoto_similarity_and_distance) between the molecular fingerprints of the two arguments (fingerprinting is currently not customizable and defaults to rdkit fingerprints with default settings). Both arguments should evaluate to a SMILES chemical structure (either string values that will be converted implicitly or a column with datatype SMILES). The expression evaluates to a score between 0 and 1 with 0 being dissimilar and 1 being entirely identically fingerprints. `tanimoto_similarity` can thus be used either directly in `order_by` clauses or as a filter when used in a relation (e.g. > 0.7). When an orderBy clause with a tanimotoSimilarity is active, the values returned for each row in the first referenced column will be augmented with the calculated similarity score (i.e. the json object that is returned for SMILES columns that is always an object with one key for the original value and one for the canonicalized SMILES string will be augmented with an additional `similarity` key that contains the numerical similarity value)

In [18]:
filter = Q.tanimoto_similarity("C=S(C)C", Q.column("SMILES")) > 0.2

chemical_dataset = api.get_published_dataset("2940e662-0ba1-4d8a-927d-03689808e99d", "latest")
benzene_similar = chemical_dataset.get_data(condition = filter)
benzene_similar

,Gene,Compound code,Replicate,log10(Concentration),Concentration unit,Compound type,Gene type,Well,Ct,SMILES,Blank,Delta Ct,Fold,Fold control,Normalized fold,Mean fold,SD fold
1,h36B4,DMSO,1,None,None,Blank,Control,A1,14.95,"{'original': 'CS(=O)C', 'canonical': 'CS(C)=O'}",15.320,0.370,1.292353,1.292353,1.000000,1.000000,0.000000
2,h36B4,DMSO,2,None,None,Blank,Control,B1,15.69,"{'original': 'CS(=O)C', 'canonical': 'CS(C)=O'}",15.320,-0.370,0.773782,0.773782,1.000000,1.000000,0.000000
9,h1A1,DMSO,1,None,None,Blank,Test,A2,37.60,"{'original': 'CS(=O)C', 'canonical': 'CS(C)=O'}",37.810,0.210,1.156688,1.292353,0.895025,1.006156,0.157163
10,h1A1,DMSO,2,None,None,Blank,Test,B2,38.02,"{'original': 'CS(=O)C', 'canonical': 'CS(C)=O'}",37.810,-0.210,0.864537,0.773782,1.117287,1.006156,0.157163
17,h1A2,DMSO,1,None,None,Blank,Test,A3,NaN,"{'original': 'CS(=O)C', 'canonical': 'CS(C)=O'}",NaN,NaN,NaN,1.292353,NaN,NaN,NaN
18,h1A2,DMSO,2,None,None,Blank,Test,B3,NaN,"{'original': 'CS(=O)C', 'canonical': 'CS(C)=O'}",NaN,NaN,NaN,0.773782,NaN,NaN,NaN
25,h1B1,DMSO,1,None,None,Blank,Test,A4,30.89,"{'original': 'CS(=O)C', 'canonical': 'CS(C)=O'}",31.475,0.585,1.500039,1.292353,1.160704,1.011125,0.211536
26,h1B1,DMSO,2,None,None,Blank,Test,B4,32.06,"{'original': 'CS(=O)C', 'canonical': 'CS(C)=O'}",31.475,-0.585,0.666649,0.773782,0.861546,1.011125,0.211536
33,h2A6,DMSO,1,None,None,Blank,Test,A5,27.83,"{'original': 'CS(=O)C', 'canonical': 'CS(C)=O'}",27.825,-0.005,0.996540,1.292353,0.771105,1.033972,0.371750
34,h2A6,DMSO,2,None,None,Blank,Test,B5,27.82,"{'original': 'CS(=O)C', 'canonical': 'CS(C)=O'}",27.825,0.005,1.003472,0.773782,1.296840,1.033972,0.371750


### Substructure search
`Q.substructure_search` let's you find chemical substructures where the first argument is searched for in the second. Both argument should evaluate to a SMILES chemical structure (either string values that will be converted implicitly or a column with datatype SMILES). `substructure_search` evaluates to a boolean value.

In [19]:
filter = Q.substructure_search("c1ccccc1", Q.column("SMILES"))

chemical_dataset = api.get_published_dataset("2940e662-0ba1-4d8a-927d-03689808e99d", "latest")
benzene_similar = chemical_dataset.get_data(condition = filter)
benzene_similar

,Gene,Compound code,Replicate,log10(Concentration),Concentration unit,Compound type,Gene type,Well,Ct,SMILES,Blank,Delta Ct,Fold,Fold control,Normalized fold,Mean fold,SD fold
3,h36B4,TCDD,1,-8,mol/L,Test,Control,C1,15.43,{'original': 'C1=C2C(=CC(=C1Cl)Cl)OC3=CC(=C(C=...,15.320,-0.110,0.926588,0.926588,1.000000,1.000000,0.000000
4,h36B4,TCDD,2,-8,mol/L,Test,Control,D1,15.43,{'original': 'C1=C2C(=CC(=C1Cl)Cl)OC3=CC(=C(C=...,15.320,-0.110,0.926588,0.926588,1.000000,1.000000,0.000000
5,h36B4,Rif,1,-4,mol/L,Test,Control,E1,15.26,{'original': 'CC1C=CC=C(C(=O)NC2=C(C(=C3C(=C2O...,15.320,0.060,1.042466,1.042466,1.000000,1.000000,0.000000
6,h36B4,Rif,2,-4,mol/L,Test,Control,F1,15.07,{'original': 'CC1C=CC=C(C(=O)NC2=C(C(=C3C(=C2O...,15.320,0.250,1.189207,1.189207,1.000000,1.000000,0.000000
11,h1A1,TCDD,1,-8,mol/L,Test,Test,C2,35.80,{'original': 'C1=C2C(=CC(=C1Cl)Cl)OC3=CC(=C(C=...,37.810,2.010,4.027822,0.926588,4.346939,4.886678,0.763306
12,h1A1,TCDD,2,-8,mol/L,Test,Test,D2,35.48,{'original': 'C1=C2C(=CC(=C1Cl)Cl)OC3=CC(=C(C=...,37.810,2.330,5.028053,0.926588,5.426417,4.886678,0.763306
13,h1A1,Rif,1,-4,mol/L,Test,Test,E2,36.74,{'original': 'CC1C=CC=C(C(=O)NC2=C(C(=C3C(=C2O...,37.810,1.070,2.099433,1.042466,2.013911,1.346107,0.944418
14,h1A1,Rif,2,-4,mol/L,Test,Test,F2,38.12,{'original': 'CC1C=CC=C(C(=O)NC2=C(C(=C3C(=C2O...,37.810,-0.310,0.806642,1.189207,0.678302,1.346107,0.944418
19,h1A2,TCDD,1,-8,mol/L,Test,Test,C3,NaN,{'original': 'C1=C2C(=CC(=C1Cl)Cl)OC3=CC(=C(C=...,NaN,NaN,NaN,0.926588,NaN,NaN,NaN
20,h1A2,TCDD,2,-8,mol/L,Test,Test,D3,NaN,{'original': 'C1=C2C(=CC(=C1Cl)Cl)OC3=CC(=C(C=...,NaN,NaN,NaN,0.926588,NaN,NaN,NaN


## Casts

The final element of the EdelweissData™ Query Language are casts, which convert data from one datatype to another. Because the query language uses a simple type system, EdelweissData is able to infer if a cast is possible. If the cast is safe and cannot fail (for example converting an integer to a string), it will be inserted automatically. If the cast is possible, but not necessarily safe (for example converting a string to an integer may fail, or converting a float to an integer will lose data), the user has to insert it explicitly using the \`cast\` function. Its first argument is the expression to cast and the second one is the datatype identifier of the type to cast to.

The possible datatypes are [listed here](/@danyx/edelweissdata-docs-create-and-publish-a-dataset#3-upload-the-schema).

In [20]:
filter = (Q.cast(Q.column("new_cases"), "xsd:integer") > 50000)

data_cast = dataset.get_data(condition = filter)

altair.Chart(data_cast).mark_circle().encode(
        x="date:T",
        y="new_cases",
        color="location",
        tooltip=["location", "date", "new_cases", "new_deaths"]
    ).properties(width=1000)

alt.Chart(...)